## Movie Review

In [1]:
from load_sentiment import load_sentiment_data
from text_utils import build_dict, generate_sequence
from models import build_lstm

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5103)


In [2]:
dirpath = "/home/ec2-user/data/rt-polaritydata/"
max_features = 10000

X_train, X_test, Y_train, Y_test, worddict = load_sentiment_data(dirpath, max_tokens=max_features)

Building dictionary.. 201209  total words  20247  unique words
Filtering to 10000  unique words


In [3]:
nb_classes = len(set(Y_train + Y_test))
print('Num. classes:', nb_classes)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

maxlen_train = max([len(x) for x in X_train])
maxlen_test  = max([len(x) for x in X_test])
print('Max length train', maxlen_train)
print('Max length tet', maxlen_test)

('Num. classes:', 2)
(9595, 'train sequences')
(1067, 'test sequences')
('Max length train', 56)
('Max length tet', 59)


In [4]:
maxlen = 60  
batch_size = 32

In [5]:
from keras.preprocessing import sequence

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
('X_train shape:', (9595, 60))
('X_test shape:', (1067, 60))


In [6]:
from keras.utils import np_utils

print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
('Y_train shape:', (9595, 2))
('Y_test shape:', (1067, 2))


In [7]:
model = build_lstm(max_features, 64, nb_classes)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, None, 64)      640000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 64)            33024       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             130         lstm_1[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2)             0           dense_1[0][0]                    
Total params: 673,154
Trainable params: 673,154
Non-trainable params: 0
___________________

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [9]:
print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=5,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 9595 samples, validate on 1067 samples
Epoch 1/5
9595/9595 [==============================] - 23s - loss: 0.6530 - acc: 0.6077 - val_loss: 0.5295 - val_acc: 0.7423
Epoch 2/5
9595/9595 [==============================] - 23s - loss: 0.4658 - acc: 0.7804 - val_loss: 0.4858 - val_acc: 0.7723
Epoch 3/5
9595/9595 [==============================] - 24s - loss: 0.3457 - acc: 0.8516 - val_loss: 0.5425 - val_acc: 0.7573
Epoch 4/5
9595/9595 [==============================] - 23s - loss: 0.2772 - acc: 0.8856 - val_loss: 0.5756 - val_acc: 0.7563
Epoch 5/5
1067/1067 [==============================] - 1s     
('Test score:', 0.56307400929223961)
('Test accuracy:', 0.74976569816344452)


In [14]:
from models import build_cnn, build_cnn_lstm, build_bidirectional_lstm

In [23]:
# %load models.py

from keras.utils import np_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input, Bidirectional

def build_lstm(max_features, embedding_dims, nb_classes):  
    model = Sequential()
    model.add(Embedding(max_features, embedding_dims, dropout=0.2))
    model.add(LSTM(embedding_dims, dropout_W=0.2, dropout_U=0.2)) 
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    return model


def build_cnn(max_features, embedding_dims, maxlen, nb_filter, filter_length, hidden_dims, nb_classes ) : 
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:
    model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
    # we use max pooling:
    model.add(MaxPooling1D(pool_length=model.output_shape[1]))

    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    return model


def build_cnn_lstm(embedding_size, maxlen, nb_filter, filter_length, pool_length, lstm_output_size, nb_classes):
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
    model.add(MaxPooling1D(pool_length=pool_length))
    model.add(LSTM(lstm_output_size))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
     
    return model     

def build_bidirectional_lstm(embedding_dims, lstm_output_size, nb_classes):
    model = Sequential()
    model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
    model.add(Bidirectional(LSTM(lstm_output_size)))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    
    return model

In [24]:
# set parameters:
embedding_dims = 128 # size of embedding dims
nb_filter = 65      # number of filters 
filter_length = 5    # 1d convolution size
hidden_dims = 250    # size of hidden layers 


print('Build model...')
cnn_model = build_cnn(max_features,embedding_dims, maxlen, nb_filter, filter_length, hidden_dims, nb_classes )

Build model...


In [25]:
cnn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 60, 128)       1280000     embedding_input_2[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 56, 65)        41665       embedding_2[0][0]                
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 1, 65)         0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 65)            0           maxpooling1d_1[0][0]             
___________________________________________________________________________________________

In [26]:
cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
nb_epoch = 5

history = cnn_model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test))

Train on 9595 samples, validate on 1067 samples
Epoch 1/5
9595/9595 [==============================] - 3s - loss: 0.0956 - acc: 0.9637 - val_loss: 0.7908 - val_acc: 0.7563
Epoch 2/5
9595/9595 [==============================] - 3s - loss: 0.0937 - acc: 0.9637 - val_loss: 0.8952 - val_acc: 0.7479
Epoch 3/5
9595/9595 [==============================] - 3s - loss: 0.0804 - acc: 0.9688 - val_loss: 0.7793 - val_acc: 0.7498
Epoch 4/5
9595/9595 [==============================] - 3s - loss: 0.0682 - acc: 0.9745 - val_loss: 0.8773 - val_acc: 0.7310
Epoch 5/5
9595/9595 [==============================] - 3s - loss: 0.0657 - acc: 0.9757 - val_loss: 0.8017 - val_acc: 0.7385
